In [2]:
import requests
import re


# получаем страницу игры по ссылке
link = 'https://www.hltv.org/matches/2342661/nexus-vs-saw-esea-advanced-season-34-europe'
request = requests.get(link).text

# функция по определению ранга команды
pattern_findrank = r"<a href=\"/ranking/teams\">#([\w-]+)<"
def get_rank(team_page):
    return re.findall(pattern_findrank, team_page)


# функция по определению тира команды
def get_tier(team_page):
    rank = get_rank(team_page)
    if len(rank) == 0:
        return 3
    else:
        if int(rank[0]) <= 15:
            return 1
        elif int(rank[0]) <=30:
            return 2
        else:
            return 3
    
    
# получаем название и страницу турнира
pattern_find_tournament = r"<div class=\"event text-ellipsis\"><a href=\"(.*)\" title=\"(.*)\""
tournament = re.findall(pattern_find_tournament, request)
tournament_link = 'https://www.hltv.org' + tournament[0][0]
tournament_name = tournament[0][1]
tournament_page = requests.get(tournament_link).text
print('Tournament:', tournament_name)


# получаем информацию о награде турнира
pattern_prize = r"<td class=\"prizepool text-ellipsis\" title=\"([\w\s$,+\-]+)\""
prize = re.findall(pattern_prize, tournament_page)[0]
print('Prize: ', prize)


# получаем тир турнира
pattern_tournament_teams = r"<a href=\"/team([\w/-]+)\" data-link-tracking-page=\"Eventpage\""
tournament_teams = re.findall(pattern_tournament_teams, tournament_page)
sum_rank = 0
for i in tournament_teams:
    page_team = requests.get('https://www.hltv.org/team' + i).text
    sum_rank += get_tier(page_team)
tournament_tier = sum_rank / len(tournament_teams)
print('Teams:', len(tournament_teams))
print('Tournament Tier:', round(tournament_tier, 1))


# получаем информацию о стадии турнира
pattern_tournament_stage = r"<div class=\"padding preformatted-text\">[\w\s]+\* ([\w\s]+).*<"
tournament_stage = re.findall(pattern_tournament_stage, request)
if len(tournament_stage) > 0:
    print('Tournament Stage:', tournament_stage[0])
print('Tournament Link:', tournament_link)

    
# получаем историю пиков и банов для начавшейся игры
pattern_history_pick = r"<div>\d\. (.*)</div>"
history_pick = re.findall(pattern_history_pick, request)
if len(history_pick) > 0:
    print('----------------------------------------------------------')
    print('History picks and bans:')
    for i in history_pick:
        print('-', i)

        
# получаем название команд 
pattern_findteam = r"class=\"teamName\">([\w\s\.-]+)<"
teams = re.findall(pattern_findteam, request)
team1 = teams[0]
team2 = teams[1]


# получаем страницы команд
pattern_team1 = r"<div class=\"team1-gradient\"><a href=\"([\w/-]+)\""
team1_link = 'https://www.hltv.org' + re.findall(pattern_team1, request)[0] 
request1 = requests.get(team1_link).text
pattern_team2 = r"<div class=\"team2-gradient\"><a href=\"([\w/-]+)\""
team2_link = 'https://www.hltv.org' + re.findall(pattern_team2, request)[0]
request2 = requests.get(team2_link).text


# получаем тир и ранг команд
rank1 = get_rank(request1)
rank2 = get_rank(request2)
tier1 = get_tier(request1)
tier2 = get_tier(request2)


# получаем информацию о командах с аналитики hltv
pattern_analytisc_hltv = r"<a href=\"(.*)\" class=\"matchpage-analytics-center-container\">"
analytisc_hltv_link = 'https://www.hltv.org' + re.findall(pattern_analytisc_hltv, request)[0]
analytisc_hltv = requests.get(analytisc_hltv_link).text
pattern_info_team1 = r"<div class=\"analytics-insights-container team1\">([\s\S]*?)<div class=\"col-6\">"
info_team1 = re.findall(pattern_info_team1, analytisc_hltv)[0]
pattern_info_team2 = r"<div class=\"analytics-insights-container team2\">([\s\S]*?)<div class=\"analytics-bookmakers g-grid\" id=\"bookmakers\">"
info_team2 = re.findall(pattern_info_team2, analytisc_hltv)[0]
pattern_find_plus = r"<div class=\"analytics-insights-indicator in-favor\"><i class=\"fa fa-plus*\"></i></div>\n\s*<div class=\"analytics-insights-info\">(.*)</div>"
find_plus1 = re.findall(pattern_find_plus, info_team1)
find_plus2 = re.findall(pattern_find_plus, info_team2)
pattern_find_minus = r"<div class=\"analytics-insights-indicator against\"><i class=\"fa fa-minus\"></i></div>\n\s*<div class=\"analytics-insights-info\">(.*)</div>"
find_minus1 = re.findall(pattern_find_minus, info_team1)
find_minus2 = re.findall(pattern_find_minus, info_team2)
print('----------------------------------------------------------')
print('Team information')
print()
print(team1)
print("Team link:", team1_link)
print('Tier:', tier1)
if len(rank1) > 0:
    print('Rank:', rank1[0])
else:
    print('Rank: -')
if len(find_plus1) != 0:
    for i in find_plus1:
        print('+', i)
else:
    print('+ No insights detected for', team1)
if len(find_minus1) != 0:
    for i in find_minus1:
        print('-', i)
else:
    print('- No insights detected for', team1)
print()
print(team2)
print("Team link:", team2_link)
print('Tier:', tier2)
if len(rank2) > 0:
    print('Rank:', rank2[0])
else:
    print('Rank: -')
if len(find_plus2) != 0:
    for i in find_plus2:
        print('+', i)
else:
    print('+ No insights detected for', team2)
if len(find_minus2) != 0:
    for i in find_minus2:
        print('-', i)
else:
    print('- No insights detected for', team2)


# получаем форму игроков с аналитики hltv
pattern_players_form1 =  r"<div class=\"analytics-head-to-head-container team1\">([\s\S]*?)<div class=\"analytics-last-matches\">"    
pattern_players_form2 =  r"<div class=\"analytics-head-to-head-container team2\">([\s\S]*?)<div class=\"analytics-last-matches\">"    
players_form1 = re.findall(pattern_players_form1, analytisc_hltv)[0]
players_form2 = re.findall(pattern_players_form2, analytisc_hltv)[0]
pattern_find_player = r"<div class=\"player-nickname\">(.*)</div>"
pattern_form_latest = r"<td class=\"table-3-months[\s\w]*?\">(.*)</td>"
pattern_form_event = r"<td class=\"table-event[\s\w]*?\"[\s\S]*?>(.*)</td>"
find_player1 = re.findall(pattern_find_player, players_form1)
form_latest1 = re.findall(pattern_form_latest, players_form1)
form_event1 = re.findall(pattern_form_event, players_form1)
find_player2 = re.findall(pattern_find_player, players_form2)
form_latest2 = re.findall(pattern_form_latest, players_form2)
form_event2 = re.findall(pattern_form_event, players_form2)
print('----------------------------------------------------------')
print('Form players (Nick | 3 months | Event)')
print()
print(team1)
i = 0
while i < len(find_player1):
    print(find_player1[i], '|', form_latest1[i], '|', form_event1[i])
    i += 1
print()
print(team2)
i = 0
while i < len(find_player2):
    print(find_player2[i], '|', form_latest2[i], '|', form_event2[i])
    i += 1


IndexError: list index out of range

In [11]:
# выбираем карту (с маленькой буквы)
maps = 'mirage'

# получаем ссылки на сыгранные карты за последние 3 месяца
pattern_stats_team = r"<a href=\"(/stats/teams/map/.*)\" class=\"moreButton\""
link_maps1 = re.findall(pattern_stats_team, request1)
link_maps2 = re.findall(pattern_stats_team, request2)


# получаем страницу с искомой картой
card_code = {'dust2':'/31/', 'mirage':'/32/', 'inferno':'/33/', 'nuke':'/34/', 'train':'/35/', 'overpass':'/40/', 'vertigo':'/46/'}
pattern_code = r"(/\d\d/)"
score_link = 'https://www.hltv.org' + link_maps1[0]
score_link1 = re.sub(pattern_code, card_code[maps], score_link)
score_link1 = score_link1.replace("amp;", "")
score_page1 = requests.get(score_link1).text
score_link = 'https://www.hltv.org' + link_maps2[0]
score_link2 = re.sub(pattern_code, card_code[maps], score_link)
score_link2 = score_link2.replace("amp;", "")
score_page2 = requests.get(score_link2).text


# получаем статистику по сыгранной карте
pattern_times_played = r"<div class=\"stats-row\"><span class=\"strong\">Times played</span><span>(\w*)<"
times_played1 = re.findall(pattern_times_played, score_page1)[0]
times_played2 = re.findall(pattern_times_played, score_page2)[0]
pattern_wins_draws_losses = r"<div class=\"stats-row\"><span class=\"strong\">Wins / draws / losses</span><span>(\w*) / (\w*) / (\w*)<"
wins_draws_losses1 = re.findall(pattern_wins_draws_losses, score_page1)
wins_draws_losses2 = re.findall(pattern_wins_draws_losses, score_page2)
pattern_total_rounds_played = r"<div class=\"stats-row\"><span class=\"strong\">Total rounds played</span><span>(\w*)<"
total_rounds_played1 = re.findall(pattern_total_rounds_played, score_page1)[0]
total_rounds_played2 = re.findall(pattern_total_rounds_played, score_page2)[0]
patter_rounds_won = r"<div class=\"stats-row\"><span class=\"strong\">Rounds won</span><span>(\w+)<"
rounds_won1 = re.findall(patter_rounds_won, score_page1)[0] 
rounds_won2 = re.findall(patter_rounds_won, score_page2)[0]
pattern_win_percent = r"<div class=\"stats-row\"><span class=\"strong\">Win percent</span><span class=\"\w*\">([\w\.%]*)<"

'''win_percent1 = re.findall(pattern_win_percent, score_page1)[0]
win_percent2 = re.findall(pattern_win_percent, score_page2)[0]'''

pattern_distribution_of_won_rounds = r"data&quot;:\[\{&quot;label&quot\;\:&quot;\w*&quot;,&quot;value&quot;:&quot;(\w*)&quot;,&quot;color&quot;:&quot;2C6EA4&quot;\},\{&quot;label&quot;:&quot;\w*&quot;,&quot;value&quot;:&quot;(\w*)&quot;"
distribution_of_won_rounds1 = re.findall(pattern_distribution_of_won_rounds, score_page1)
distribution_of_won_rounds2 = re.findall(pattern_distribution_of_won_rounds, score_page2)

'''if int(total_rounds_played1) != 0:
    win_round_percent1 = str(round((int(rounds_won1) / int(total_rounds_played1) * 100), 1)) + '%'
    win_ct_1 = str(round((int(distribution_of_won_rounds1[0][0]) / (int(distribution_of_won_rounds1[0][0]) + int(distribution_of_won_rounds1[0][1])) * 100), 2)) + '%' 
    win_t_1 = str(round((int(distribution_of_won_rounds1[0][1]) / (int(distribution_of_won_rounds1[0][0]) + int(distribution_of_won_rounds1[0][1])) * 100), 2)) + '%' 
else:
    win_round_percent1 = 0
    win_ct_1 = 0
    win_t_1 = 0'''
    
'''if int(total_rounds_played2) != 0:
    win_round_percent2 = str(round((int(rounds_won2) / int(total_rounds_played2) * 100), 1)) + '%'
    win_ct_2 = str(round((int(distribution_of_won_rounds2[0][0]) / (int(distribution_of_won_rounds2[0][0]) + int(distribution_of_won_rounds2[0][1])) * 100), 2)) + '%' 
    win_t_2 = str(round((int(distribution_of_won_rounds2[0][1]) / (int(distribution_of_won_rounds2[0][0]) + int(distribution_of_won_rounds2[0][1])) * 100), 2)) + '%' 
else:
    win_round_percent2 = 0
    win_ct_2 = 0
    win_t_2 = 0'''
    
pattern_pistol_rounds = r"<div class=\"stats-row\"><span class=\"strong\">Pistol rounds</span><span>(\w*)<"
pistol_rounds1 = re.findall(pattern_pistol_rounds, score_page1)[0]
pistol_rounds2 = re.findall(pattern_pistol_rounds, score_page2)[0]
pattern_pistol_rounds_won = r"<div class=\"stats-row\"><span class=\"strong\">Pistol rounds won</span><span>(\w*)<"
pistol_rounds_won1 = re.findall(pattern_pistol_rounds_won, score_page1)[0]
pistol_rounds_won2 = re.findall(pattern_pistol_rounds_won, score_page2)[0]
pattern_pistol_round_win_percent = r"<div class=\"stats-row\"><span class=\"strong\">Pistol round win percent</span><span class=\"\w*\">([\w\.%]*)<"

'''pistol_round_win_percent1 = re.findall(pattern_pistol_round_win_percent, score_page1)[0]
pistol_round_win_percent2 = re.findall(pattern_pistol_round_win_percent, score_page2)[0]'''

# получаем статистику сыгранных матчей на выбранной карте
pattern_match_date = r"<td class=\"time\"><a href=\".*\">([\d/]*)<"
match_date1 = re.findall(pattern_match_date, score_page1)
match_date2 = re.findall(pattern_match_date, score_page2)
pattern_score_match = r"<td class=\"statsTeamMapResult match-(\w*)\">(.*)</td>"
score_match1 = re.findall(pattern_score_match, score_page1)
score_match2 = re.findall(pattern_score_match, score_page2)
pattern_opponent = r"<td><a href=\".*\" class=\"image-and-label\".*><span>(.*)</span></a>"
opponent1 = re.findall(pattern_opponent, score_page1)
opponent2 = re.findall(pattern_opponent, score_page2)
pattern_opponent_link = r"<td><a href=\"/stats/teams/(\d*)/.*?\" class=\"image-and-label\" data-tooltip-id=\".*?\">"
opponent_link1 = re.findall(pattern_opponent_link, score_page1)
opponent_link2 = re.findall(pattern_opponent_link, score_page2)
ranks_opponent1 = []
ranks_opponent2 = []
i = 0
while i < len(opponent1):
    opponent = opponent1[i].lower()
    opponent = opponent.replace(' ', '-')
    page_opponent = requests.get('https://www.hltv.org/team/' + opponent_link1[i] + '/opponent').text
    rank = get_rank(page_opponent)
    if len(rank) == 0:
        ranks_opponent1.append('-')
    else:
        ranks_opponent1.append(rank[0])
    i += 1
i = 0
while i < len(opponent2):
    opponent = opponent2[i].lower()
    opponent = opponent.replace(' ', '-')
    page_opponent = requests.get('https://www.hltv.org/team/' + opponent_link2[i] + '/opponent').text
    rank = get_rank(page_opponent)
    if len(rank) == 0:
        ranks_opponent2.append('-')
    else:
        ranks_opponent2.append(rank[0])
    i += 1
pattern_match_tournament = r"<td class=\"\"><a href=\".*\" class=\"image-and-label\" data-tooltip-id=\".*\" title=\"(.*)\">"
match_tournament1 = re.findall(pattern_match_tournament, score_page1)
match_tournament2 = re.findall(pattern_match_tournament, score_page2)

print('----------------------------------------------------------')
print(team1)
print(maps, 'stats')
print('Times played:', times_played1)
print('Wins/draws/losses:', wins_draws_losses1[0][0], wins_draws_losses1[0][1], wins_draws_losses1[0][2])
print('Total rounds played:', total_rounds_played1)
print('Rounds won:', rounds_won1)
print('Win percent:', win_percent1)
print('Win round percent:', win_round_percent1)
print('Win CT percent:', win_ct_1)
print('Win T percent:', win_t_1)
print('Pistol rounds:', pistol_rounds1)
print('Pistol rounds won:', pistol_rounds_won1)
print('Pistol round win percent:', pistol_round_win_percent1)

print()
print(team2)
print(maps, 'stats')
print('Times played:', times_played2)
print('Wins/draws/losses:', wins_draws_losses2[0][0], wins_draws_losses2[0][1], wins_draws_losses2[0][2])
print('Total rounds played:', total_rounds_played2)
print('Rounds won:', rounds_won2)
print('Win percent:', win_percent2)
print('Win round percent:', win_round_percent2)
print('Win CT percent:', win_ct_2)
print('Win T percent:', win_t_2)
print('Pistol rounds:', pistol_rounds2)
print('Pistol rounds won:', pistol_rounds_won2)
print('Pistol round win percent:', pistol_round_win_percent2)

print('----------------------------------------------------------')
print('Mathes')
print(team1)
i = 0
while i < len(match_date1):
    print(match_date1[i], '|', score_match1[i][0], '|', score_match1[i][1], '| Rank:', ranks_opponent1[i], opponent1[i], '|', match_tournament1[i])
    i += 1
i = 0
print()
print(team2)
while i < len(match_date2):
    print(match_date2[i], '|', score_match2[i][0], '|', score_match2[i][1], '| Rank:', ranks_opponent2[i], opponent2[i], '|', match_tournament2[i])
    i += 1

----------------------------------------------------------
Spirit
mirage stats
Times played: 14
Wins/draws/losses: 11 0 3
Total rounds played: 367
Rounds won: 210
Win percent: 78.6%
Win round percent: 57.2%
Win CT percent: 51.43%
Win T percent: 48.57%
Pistol rounds: 28
Pistol rounds won: 18
Pistol round win percent: 64.3%

GamerLegion
mirage stats
Times played: 12
Wins/draws/losses: 9 0 3
Total rounds played: 340
Rounds won: 186
Win percent: 75.0%
Win round percent: 54.7%
Win CT percent: 51.61%
Win T percent: 48.39%
Pistol rounds: 24
Pistol rounds won: 12
Pistol round win percent: 50.0%
----------------------------------------------------------
Mathes
Spirit
16/02/20 | won | 16 - 14 | Rank: 37 Gambit Youngsters | FLASHPOINT Europe Closed Qualifier
16/02/20 | won | 16 - 4 | Rank: 30 Copenhagen Flames | FLASHPOINT Europe Closed Qualifier
16/02/20 | won | 16 - 12 | Rank: 46 GamerLegion | FLASHPOINT Europe Closed Qualifier
15/02/20 | won | 16 - 11 | Rank: 47 Juggernauts | FLASHPOINT Europe

# Начало запуска!

Очередь:


In [24]:
import requests
import re
import numpy as np
import pandas as pd
import time

link = 'https://www.hltv.org/matches/2345218/heroic-vs-vitality-iem-beijing-haidian-2020-europe'

# получаем страницу
def get_request(link):
    headers_initial = {
        'user-agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-language': 'en-US,en;q=0.9',
        'cache-control': 'no-cache',
        'upgrade-insecure-requests': '1',
    } 
    return requests.get(link, headers=headers_initial).text

# получаем названия команд
def get_team_names(request):
    pattern_findteam = r"class=\"teamName\">([\w\s\.-]+)<"
    team_names = re.findall(pattern_findteam, request)
    return team_names[0], team_names[1]
    
# получаем страницы команд
def get_team_pages(request):
    pattern_team1 = r"<div class=\"team1-gradient\"><a href=\"([\w/-]+)\""
    team1_link = 'https://www.hltv.org' + re.findall(pattern_team1, request)[0] 
    pattern_team2 = r"<div class=\"team2-gradient\"><a href=\"([\w/-]+)\""
    team2_link = 'https://www.hltv.org' + re.findall(pattern_team2, request)[0]
    return get_request(team1_link), get_request(team2_link)

# функция по определению ранга команды
def get_rank(team_page):
    pattern_findrank = r"<a href=\"/ranking/teams\">#([\w-]+)<"
    return re.findall(pattern_findrank, team_page)[0]

# получаем страницу аналитики
def get_analytics_page(request):
    pattern_analytics_hltv = r"<a href=\"(.*)\" class=\"matchpage-analytics-center-container\">"
    analytics_hltv_link = 'https://www.hltv.org' + re.findall(pattern_analytics_hltv, request)[0]
    return get_request(analytics_hltv_link)

# получаем формы игроков двух команд
def get_players_forms(analytics_page):
    pattern_players_form1 =  r"<div class=\"analytics-head-to-head-container team1\">([\s\S]*?)<div class=\"analytics-last-matches\">"    
    players_form1 = re.findall(pattern_players_form1, analytics_page)[0]
    pattern_players_form2 =  r"<div class=\"analytics-head-to-head-container team2\">([\s\S]*?)<div class=\"analytics-last-matches\">"
    players_form2 = re.findall(pattern_players_form2, analytics_page)[0]
    return players_form1, players_form2

# получаем форму команды за 3 месяца
def get_form_latest(players_form):
    pattern_form_latest = r"<td class=\"table-3-months[\s\w]*?\">(.*)</td>"
    form_latest = list()
    for i in re.findall(pattern_form_latest, players_form):
        form_latest.append(float(i))
    return form_latest

# получаем форму команды на текущем турнире
def get_form_event(players_form):
    pattern_form_event = r"<td class=\"table-event[\s\w]*?\"[\s\S]*?>(.*)</td>"
    if re.findall(pattern_form_event, players_form)[0] == '-':
        return get_form_latest(players_form)
    else:
        form_event = list()
        for i in re.findall(pattern_form_event, players_form):
            form_event.append(float(i))
        return(form_event)

# получаем статистики формы команды
def get_form_statistics(list_form):
    avg_form = sum(list_form) / len(list_form)
    form_range = max(list_form) - min(list_form)
    diffs = 0
    for n in list_form:
        diffs += (n - avg_form)**(2)
    sd_form = (diffs/(len(list_form)-1))**(0.5)
    return avg_form, form_range, sd_form

# получаем статистику по раундам в игре
def get_avg_rounds(analytics_page):
    match_pattern =  r"<div class=\"analytics-handicap-map-data-table-container\">([\s\S]*?)</table>"    
    matchs = re.findall(match_pattern, analytics_page)
    return matchs[0], matchs[1]

# получаем словарь с названиями карт и средние раунды при победах и поражениях
def get_avg_rounds_maps(avg_rounds):
    map_pattern = r"<tr>([\s\S]*?)</tr>"
    maps = re.findall(map_pattern, avg_rounds)
    map_name_pattern = r"<div class=\"mapname\">([\s\S]*?)</div>"
    avg_rounds_pattern = r"<td class=\"analytics-handicap-map-data-avg\">([\s\S]*?)</td>"
    maps_name = {}
    for i in maps:
        if len(re.findall(map_name_pattern, i)) > 0:
            maps_name[re.findall(map_name_pattern, i)[0]] = re.findall(avg_rounds_pattern, i)
    return maps_name

# получаем страницу с искомой картой (с мелкой буквы)
def score_page(maps_name, team_page):
    pattern_stats_team = r"<a href=\"(/stats/teams/map/.*)\" class=\"moreButton\""
    link_maps = re.findall(pattern_stats_team, team_page)
    card_code = {'dust2':'/31/', 'mirage':'/32/', 'inferno':'/33/', 'nuke':'/34/', 'train':'/35/', 'overpass':'/40/', 'vertigo':'/46/'}
    pattern_code = r"(/\d\d/)"
    link = 'https://www.hltv.org' + link_maps[0]
    score_link = re.sub(pattern_code, card_code[maps_name], link)
    score_link = score_link.replace("amp;", "")
    score_page = get_request(score_link)
    return score_page

# получаем процент выигранных игр на карте
def win_percent_map(score_page):
    pattern_win_percent = r"<div class=\"stats-row\"><span class=\"strong\">Win percent</span><span class=\"\w*\">([\w\.%]*)<"
    if len(re.findall(pattern_win_percent, score_page)) > 0:
        win_percent = re.findall(pattern_win_percent, score_page)[0].replace('%','')
    else:
        win_percent = 0
    return float(win_percent)

# получаем процент выигранных пистолеток
def win_pistol_rounds(score_page):
    pattern_pistol_round_win_percent = r"<div class=\"stats-row\"><span class=\"strong\">Pistol round win percent</span><span class=\"\w*\">([\w\.%]*)<"
    if len(re.findall(pattern_pistol_round_win_percent, score_page)) > 0:
        pistol_round_win_percent = re.findall(pattern_pistol_round_win_percent, score_page)[0].replace('%','')
    else:
        pistol_round_win_percent = 0
    return float(pistol_round_win_percent)

# получаем распределение долей общих выигранных карт, за кт и за т
def distribution_of_won_rounds(score_page):
    pattern_total_rounds_played = r"<div class=\"stats-row\"><span class=\"strong\">Total rounds played</span><span>(\w*)<"
    if len(re.findall(pattern_total_rounds_played, score_page)) > 0:
        total_rounds_played = re.findall(pattern_total_rounds_played, score_page)[0]
    else:
        total_rounds_played = 0
    patter_rounds_won = r"<div class=\"stats-row\"><span class=\"strong\">Rounds won</span><span>(\w+)<"
    if len(re.findall(patter_rounds_won, score_page)) > 0:
        rounds_won = re.findall(patter_rounds_won, score_page)[0]
    else:
        rounds_won = 0
    pattern_distribution_of_won_rounds = r"data&quot;:\[\{&quot;label&quot\;\:&quot;\w*&quot;,&quot;value&quot;:&quot;(\w*)&quot;,&quot;color&quot;:&quot;2C6EA4&quot;\},\{&quot;label&quot;:&quot;\w*&quot;,&quot;value&quot;:&quot;(\w*)&quot;"
    distribution_of_won_rounds = re.findall(pattern_distribution_of_won_rounds, score_page)
    if int(total_rounds_played) != 0:
        win_round_percent = round((int(rounds_won) / int(total_rounds_played) * 100), 1)
        win_ct = round((int(distribution_of_won_rounds[0][0]) / (int(distribution_of_won_rounds[0][0]) + int(distribution_of_won_rounds[0][1])) * 100), 2)
        win_t = round((int(distribution_of_won_rounds[0][1]) / (int(distribution_of_won_rounds[0][0]) + int(distribution_of_won_rounds[0][1])) * 100), 2)
    else:
        win_round_percent = 0
        win_ct = 0
        win_t = 0
    return [win_round_percent, win_ct, win_t]

# формирование словаря с информацией по карте
def creature_info(map_name, team_page):
    d = {}
    score = score_page(map_name.lower(), team_page)
    l = distribution_of_won_rounds(score)
    l.append(win_percent_map(score))
    l.append(win_pistol_rounds(score))
    d[map_name] = l
    return d
    
# формирование датафрейма с парамтерами для карты map_name
def creature_df(map_name):
    d = {'link': [link], 'rounds': [0], 'win': [0],
         'team1_rank': [team1_rank], 'team1_form_avg_latest': [team1_form_avg_latest], 'team1_form_range_latest': [team1_form_range_latest], 
         'team1_form_sd_latest': [team1_form_sd_latest], 'team1_form_avg_event': [team1_form_avg_event], 'team1_form_range_event': [team1_form_range_event], 
         'team1_form_sd_event': [team1_form_sd_event], 'team1_win_round_percent': team1_map_info[map_name][0], 
         'team1_win_ct': team1_map_info[map_name][1], 'team1_win_t': team1_map_info[map_name][2],
         'team1_win_percent_map': team1_map_info[map_name][3], 'team1_win_pistol_rounds': team1_map_info[map_name][4],
         'team1_lost_in_wins': [float(team1_avg_rounds[map_name][0].replace('-', '0'))], 
         'team1_won_in_losses': [float(team1_avg_rounds[map_name][1].replace('-', '0'))], 
         'team2_rank': [team2_rank], 'team2_form_avg_latest': [team2_form_avg_latest], 'team2_form_range_latest': [team2_form_range_latest], 
         'team2_form_sd_latest': [team2_form_sd_latest], 'team2_form_avg_event': [team2_form_avg_event], 'team2_form_range_event': [team2_form_range_event], 
         'team2_form_sd_event': [team2_form_sd_event], 'team2_win_round_percent': team2_map_info[map_name][0], 
         'team2_win_ct': team2_map_info[map_name][1], 'team2_win_t': team2_map_info[map_name][2],
         'team2_win_percent_map': team2_map_info[map_name][3], 'team2_win_pistol_rounds': team2_map_info[map_name][4],
         'team2_lost_in_wins': [float(team2_avg_rounds[map_name][0].replace('-', '0'))], 
         'team2_won_in_losses': [float(team2_avg_rounds[map_name][1].replace('-', '0'))]}
    df = pd.DataFrame(d)
    return df

# перезапись датафрейма
def rewrite_csv(map_name):
    df1 = pd.read_csv(map_name+'.csv')
    df2 = creature_df(map_name)
    df = pd.concat([df1, df2])
    df.to_csv(map_name+'.csv', index=False)

# получаем страницы матча и страницы команд
request = get_request(link)
team1_page, team2_page = get_team_pages(request)

# названия команд
team1, team2 = get_team_names(request)

# ранги команд
team1_rank = int(get_rank(team1_page))
team2_rank = int(get_rank(team2_page))

# получаем страницу аналитики и формы игроков команд
analytics_page = get_analytics_page(request)
players_form1, players_form2 = get_players_forms(analytics_page)

# статистики (среднее, размах, отклонение) команд за последние 3 месяца
team1_form_latest = get_form_latest(players_form1)
team2_form_latest = get_form_latest(players_form2)
team1_form_avg_latest, team1_form_range_latest, team1_form_sd_latest = get_form_statistics(team1_form_latest)
team2_form_avg_latest, team2_form_range_latest, team2_form_sd_latest = get_form_statistics(team2_form_latest)

# статистики (среднее, размах, отклонение) команд на текущем турнире
team1_form_event = get_form_event(players_form1)
team2_form_event = get_form_event(players_form2)
team1_form_avg_event, team1_form_range_event, team1_form_sd_event = get_form_statistics(team1_form_event)
team2_form_avg_event, team2_form_range_event, team2_form_sd_event = get_form_statistics(team2_form_event)

# получаем статистику команд по сыгранным раундам 
avg_rounds1, avg_rounds2 = get_avg_rounds(analytics_page)
team1_avg_rounds = get_avg_rounds_maps(avg_rounds1)
team2_avg_rounds = get_avg_rounds_maps(avg_rounds2)

'''# инфа на картах
team1_map_info = creature_info(team1_page)
team2_map_info = creature_info(team2_page)'''

'''df_dust2 = creature_df('Dust2')
df_mirage = creature_df('Mirage')
df_inferno = creature_df('Inferno')
df_nuke = creature_df('Nuke')
df_train = creature_df('Train')
df_overpass = creature_df('Overpass')
df_vertigo = creature_df('Vertigo')

df_dust2.to_csv('Dust2.csv', index=False)
df_mirage.to_csv('Mirage.csv', index=False)
df_inferno.to_csv('Inferno.csv', index=False)
df_nuke.to_csv('Nuke.csv', index=False)
df_train.to_csv('Train.csv', index=False)
df_overpass.to_csv('Overpass.csv', index=False)
df_vertigo.to_csv('Vertigo.csv', index=False)'''
    
print('Все ок')


Все ок


In [25]:
for i in ['Dust2', 'Mirage', 'Inferno', 'Nuke', 'Train', 'Overpass', 'Vertigo']:
    team1_map_info = creature_info(i, team1_page)
    team2_map_info = creature_info(i, team2_page)
    rewrite_csv(i)
    df = pd.read_csv(i + '.csv')
    #print(df)
    time.sleep(100)
    print(i, '- готово')
print('Парсинг завершен!')

Dust2 - готово
Mirage - готово
Inferno - готово
Nuke - готово
Train - готово
Overpass - готово
Vertigo - готово
Парсинг завершен!


# Конец запуска!

In [11]:
df = pd.read_csv('Vertigo.csv')
print(df)

                                                link  rounds  win  team1_rank  \
0  https://www.hltv.org/matches/2342712/gambit-yo...      48    2          31   
1  https://www.hltv.org/matches/2342422/geng-vs-l...      25    1          18   
2  https://www.hltv.org/matches/2342748/saw-vs-no...      30    2          43   
3  https://www.hltv.org/matches/2342814/gambit-yo...       0    0          30   
4  https://www.hltv.org/matches/2342815/endpoint-...       0    0          36   

   team1_form_avg_latest  team1_form_range_latest  team1_form_sd_latest  \
0                  1.084                     0.30              0.132401   
1                  1.006                     0.16              0.069857   
2                  1.080                     0.16              0.065955   
3                  1.088                     0.29              0.132174   
4                  1.076                     0.19              0.075366   

   team1_form_avg_event  team1_form_range_event  team1_form_sd

Запись в датафрейм

In [75]:
team1_map_info = creature_info('Dust2', team1_page)
team2_map_info = creature_info('Dust2', team2_page)
df_dust2 = creature_df('Dust2')
df_dust2.to_csv('Dust2.csv', index=False)
df = pd.read_csv('Dust2.csv')
df

,link,rounds,win,team1_rank,team1_form_avg_latest,team1_form_range_latest,team1_form_sd_latest,team1_form_avg_event,team1_form_range_event,team1_form_sd_event,...,team2_form_avg_event,team2_form_range_event,team2_form_sd_event,team2_win_round_percent,team2_win_ct,team2_win_t,team2_win_percent_map,team2_win_pistol_rounds,team2_lost_in_wins,team2_won_in_losses
0,https://www.hltv.org/matches/2342568/nordavind...,0,0,41,1.042,0.06,0.023875,1.042,0.06,0.023875,...,1.05,0.25,0.089443,54.2,41.67,58.33,50.0,50.0,8.5,11.2


In [76]:
team1_map_info = creature_info('Mirage', team1_page)
team2_map_info = creature_info('Mirage', team2_page)
df_dust2 = creature_df('Mirage')
df_dust2.to_csv('Mirage.csv', index=False)
df = pd.read_csv('Mirage.csv')
df

,link,rounds,win,team1_rank,team1_form_avg_latest,team1_form_range_latest,team1_form_sd_latest,team1_form_avg_event,team1_form_range_event,team1_form_sd_event,...,team2_form_avg_event,team2_form_range_event,team2_form_sd_event,team2_win_round_percent,team2_win_ct,team2_win_t,team2_win_percent_map,team2_win_pistol_rounds,team2_lost_in_wins,team2_won_in_losses
0,https://www.hltv.org/matches/2342568/nordavind...,0,0,41,1.042,0.06,0.023875,1.042,0.06,0.023875,...,1.05,0.25,0.089443,44.4,54.97,45.03,18.8,50.0,10.33,10.2


In [77]:
team1_map_info = creature_info('Inferno', team1_page)
team2_map_info = creature_info('Inferno', team2_page)
df_dust2 = creature_df('Inferno')
df_dust2.to_csv('Inferno.csv', index=False)
df = pd.read_csv('Inferno.csv')
df

,link,rounds,win,team1_rank,team1_form_avg_latest,team1_form_range_latest,team1_form_sd_latest,team1_form_avg_event,team1_form_range_event,team1_form_sd_event,...,team2_form_avg_event,team2_form_range_event,team2_form_sd_event,team2_win_round_percent,team2_win_ct,team2_win_t,team2_win_percent_map,team2_win_pistol_rounds,team2_lost_in_wins,team2_won_in_losses
0,https://www.hltv.org/matches/2342568/nordavind...,0,0,41,1.042,0.06,0.023875,1.042,0.06,0.023875,...,1.05,0.25,0.089443,51.1,49.44,50.56,57.1,35.7,8.67,8.33


In [78]:
team1_map_info = creature_info('Nuke', team1_page)
team2_map_info = creature_info('Nuke', team2_page)
df_dust2 = creature_df('Nuke')
df_dust2.to_csv('Nuke.csv', index=False)
df = pd.read_csv('Nuke.csv')
df

,link,rounds,win,team1_rank,team1_form_avg_latest,team1_form_range_latest,team1_form_sd_latest,team1_form_avg_event,team1_form_range_event,team1_form_sd_event,...,team2_form_avg_event,team2_form_range_event,team2_form_sd_event,team2_win_round_percent,team2_win_ct,team2_win_t,team2_win_percent_map,team2_win_pistol_rounds,team2_lost_in_wins,team2_won_in_losses
0,https://www.hltv.org/matches/2342568/nordavind...,0,0,41,1.042,0.06,0.023875,1.042,0.06,0.023875,...,1.05,0.25,0.089443,50.5,54.25,45.75,52.9,55.9,9.2,8.0


In [79]:
team1_map_info = creature_info('Train', team1_page)
team2_map_info = creature_info('Train', team2_page)
df_dust2 = creature_df('Train')
df_dust2.to_csv('Train.csv', index=False)
df = pd.read_csv('Train.csv')
df

,link,rounds,win,team1_rank,team1_form_avg_latest,team1_form_range_latest,team1_form_sd_latest,team1_form_avg_event,team1_form_range_event,team1_form_sd_event,...,team2_form_avg_event,team2_form_range_event,team2_form_sd_event,team2_win_round_percent,team2_win_ct,team2_win_t,team2_win_percent_map,team2_win_pistol_rounds,team2_lost_in_wins,team2_won_in_losses
0,https://www.hltv.org/matches/2342568/nordavind...,0,0,41,1.042,0.06,0.023875,1.042,0.06,0.023875,...,1.05,0.25,0.089443,49.0,55.09,44.91,50.0,62.5,11.5,11.75


In [80]:
team1_map_info = creature_info('Overpass', team1_page)
team2_map_info = creature_info('Overpass', team2_page)
df_dust2 = creature_df('Overpass')
df_dust2.to_csv('Overpass.csv', index=False)
df = pd.read_csv('Overpass.csv')
df

,link,rounds,win,team1_rank,team1_form_avg_latest,team1_form_range_latest,team1_form_sd_latest,team1_form_avg_event,team1_form_range_event,team1_form_sd_event,...,team2_form_avg_event,team2_form_range_event,team2_form_sd_event,team2_win_round_percent,team2_win_ct,team2_win_t,team2_win_percent_map,team2_win_pistol_rounds,team2_lost_in_wins,team2_won_in_losses
0,https://www.hltv.org/matches/2342568/nordavind...,0,0,41,1.042,0.06,0.023875,1.042,0.06,0.023875,...,1.05,0.25,0.089443,44.2,54.0,46.0,25.0,50.0,15.0,11.5


In [81]:
team1_map_info = creature_info('Vertigo', team1_page)
team2_map_info = creature_info('Vertigo', team2_page)
df_dust2 = creature_df('Vertigo')
df_dust2.to_csv('Vertigo.csv', index=False)
df = pd.read_csv('Vertigo.csv')
df

,link,rounds,win,team1_rank,team1_form_avg_latest,team1_form_range_latest,team1_form_sd_latest,team1_form_avg_event,team1_form_range_event,team1_form_sd_event,...,team2_form_avg_event,team2_form_range_event,team2_form_sd_event,team2_win_round_percent,team2_win_ct,team2_win_t,team2_win_percent_map,team2_win_pistol_rounds,team2_lost_in_wins,team2_won_in_losses
0,https://www.hltv.org/matches/2342568/nordavind...,0,0,41,1.042,0.06,0.023875,1.042,0.06,0.023875,...,1.05,0.25,0.089443,53.5,46.61,53.39,71.4,60.7,11.17,13.75


In [1]:
a = [1, 2, 3]
b = [i*i for i in a]
b

[1, 4, 9]

In [14]:
def gen(count):
    for i in range(1,count+1):
        yield i*i

b = gen(5)
for i in b:
    print(i)
    
c = (i*i for i in range(1,6))
next(c)
next(c)

1
4
9
16
25


4

# Предсказания

In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Классификация

## Инферно

### Загрузка обучающей выборки

In [26]:
df = pd.read_csv('Inferno_train.csv')
x_train = df.drop(['link', 'rounds', 'win'], axis=1)
y_train = df.win
y_train = y_train.astype('int')

### Загрузка тестовой выборки

In [27]:
df_test = pd.read_csv('Inferno_test.csv')
x_test = df_test.drop(['link', 'rounds', 'win'], axis=1)

### Выбираем лучшие параметры

In [28]:
rf = RandomForestClassifier(random_state=0)
parametrs = {'n_estimators': range(10, 50, 10),
             'max_depth': range(1, 12, 2),
             'min_samples_leaf': range(1, 7),
             'min_samples_split': range(2, 9, 2)}
gsrd = GridSearchCV(rf, parametrs, cv=3, n_jobs=-1)
gsrd.estimator
gsrd.fit(x_train, y_train)
gsrd.best_estimator_.fit(x_train, y_train)

C:\Users\Kereat\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=3, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

### Получаем предсказание

In [29]:
gsrd.predict(x_test)[0]

2

### Делаем предсказания на основе 1000 прогонов

In [30]:
i = 0
r = 0
while i < 1000:
    rf = RandomForestClassifier(n_estimators=20, max_depth=3, min_samples_leaf=2, min_samples_split=2)
    rf.fit(x_train, y_train)
    predictions = rf.predict(x_test)
    i += 1
    r += predictions[0]
print(r / 1000)

1.542


# Mirage

### Загрузка обучающей выборки

In [31]:
df = pd.read_csv('Mirage_train.csv')
x_train = df.drop(['link', 'rounds', 'win'], axis=1)
y_train = df.win
y_train = y_train.astype('int')

### Загрузка тестовой выборки

In [32]:
df_test = pd.read_csv('Mirage_test.csv')
x_test = df_test.drop(['link', 'rounds', 'win'], axis=1)

### Выбираем лучшие параметры

In [33]:
rf = RandomForestClassifier(random_state=0)
parametrs = {'n_estimators': range(10, 50, 10),
             'max_depth': range(1, 12, 2),
             'min_samples_leaf': range(1, 7),
             'min_samples_split': range(2, 9, 2)}
gsrd = GridSearchCV(rf, parametrs, cv=3, n_jobs=-1)
gsrd.estimator
gsrd.fit(x_train, y_train)
gsrd.best_estimator_.fit(x_train, y_train)

C:\Users\Kereat\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Kereat\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=3, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

### Получаем предсказание

In [34]:
gsrd.predict(x_test)[0]

2

### Делаем предсказания на основе 1000 прогонов

In [35]:
i = 0
r = 0
while i < 1000:
    rf = RandomForestClassifier(n_estimators=10, max_depth=3, min_samples_leaf=1, min_samples_split=2)
    rf.fit(x_train, y_train)
    predictions = rf.predict(x_test)
    i += 1
    r += predictions[0]
print(r / 1000)

1.585


# Регрессия

## Инферно

### Загрузка обучающей выборки

In [36]:
df = pd.read_csv('Inferno_train.csv')
x_train = df.drop(['link', 'rounds', 'win'], axis=1)
y_train = df.rounds

### Загрузка тестовой выборки

In [37]:
df_test = pd.read_csv('Inferno_test.csv')
x_test = df_test.drop(['link', 'rounds', 'win'], axis=1)

### Выбираем лучшие параметры

In [38]:
rf = RandomForestRegressor(random_state=0)
parametrs = {'n_estimators': range(10, 50, 10),
             'max_depth': range(1, 12, 2),
             'min_samples_leaf': range(1, 7),
             'min_samples_split': range(2, 9, 2)}
gsrd = GridSearchCV(rf, parametrs, cv=3, n_jobs=-1)
gsrd.estimator
gsrd.fit(x_train, y_train)
gsrd.best_estimator_.fit(x_train, y_train)

C:\Users\Kereat\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=1,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=30,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

### Получаем предсказание

In [39]:
gsrd.predict(x_test)[0] * 16

10.914890246455284

### Делаем предсказания на основе 1000 прогнозов

In [40]:
i = 0
r = 0
while i < 1000:
    rf = RandomForestRegressor(n_estimators=30, max_depth=1, min_samples_leaf=4, min_samples_split=2)
    rf.fit(x_train, y_train)
    predictions = rf.predict(x_test)
    i += 1
    r += predictions[0]
print(r / 1000 * 16)

10.828230014213712


## Mirage

### Загрузка обучающей выборки

In [41]:
df = pd.read_csv('Mirage_train.csv')
x_train = df.drop(['link', 'rounds', 'win'], axis=1)
y_train = df.rounds

### Загрузка тестовой выборки

In [42]:
df_test = pd.read_csv('Mirage_train.csv')
x_test = df_test.drop(['link', 'rounds', 'win'], axis=1)

### Выбираем лучшие параметры

In [43]:
rf = RandomForestRegressor(random_state=0)
parametrs = {'n_estimators': range(10, 50, 10),
             'max_depth': range(1, 12, 2),
             'min_samples_leaf': range(1, 7),
             'min_samples_split': range(2, 9, 2)}
gsrd = GridSearchCV(rf, parametrs, cv=3, n_jobs=-1)
gsrd.estimator
gsrd.fit(x_train, y_train)
gsrd.best_estimator_.fit(x_train, y_train)

C:\Users\Kereat\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=1,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=6, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=40,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

### Получаем предсказание

In [44]:
gsrd.predict(x_test)[0] * 16

9.965267638573575

### Делаем предсказания на основе 1000 прогнозов

In [45]:
i = 0
r = 0
while i < 1000:
    rf = RandomForestRegressor(n_estimators=30, max_depth=1, min_samples_leaf=4, min_samples_split=2)
    rf.fit(x_train, y_train)
    predictions = rf.predict(x_test)
    i += 1
    r += predictions[0]
print(r / 1000 * 16)

10.253742426512192
